In [ ]:
# import library
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
# create ticker object and fetch historical market data
def ticker_history(comp, dt):
    ticker = yf.Ticker(comp)
    df = ticker.history(period=dt)
    return df

In [ ]:
# split data into training and test data
def train_test_split(df, tSplit, col):
    train = df.loc[:f'{tSplit}', col].values
    test = df.loc[f'{tSplit+1}':, col].values
    return train, test

In [ ]:
# standardize training data
def scale_fit_transform(train, scaler):
    train = train.reshape(-1, 1)
    trainStd = scaler.fit_transform(train)
    return trainStd

In [ ]:
# convert training data to inputs and outputs
def split_time_series(seq, step):
    X, y = list(), list()
    for i in range(len(seq)):
        end = i + step
        if end > len(seq) - 1:
            break
        xSeq, ySeq = seq[i:end], seq[end]
        X.append(xSeq)
        y.append(ySeq)
    return np.array(X), np.array(y)

In [ ]:
# create LSTM model
def model_lstm(step, feature):
    lstm = Sequential()
    lstm.add(LSTM(units=125, activation='tanh', input_shape=(step, feature)))
    lstm.add(Dense(units=1))
    lstm.compile(optimizer='RMSprop', loss='mse')
    return lstm

In [ ]:
# transform test data into samples and inverse transform predictions
def predict_lstm(df, lstm, test, XTest, yTest, col, step):
    dfTot = df.loc[:,col]
    dIn = dfTot[len(dfTot) - len(test) - step :].values
    dIn = dIn.reshape(-1, 1)
    dIn = scaler.transform(dIn)
    XTest, yTest = split_time_series(dIn, step)
    XTest = XTest.reshape(XTest.shape[0], XTest.shape[1], feature)
    yPred = lstm.predict(XTest)
    yPred = scaler.inverse_transform(yPred)
    return XTest, yTest, yPred

In [ ]:
comp = ['NVDA', 'AMD']
dt = '5y'
col = 'High'
tSplit = 2023
step = 60
feature = 1
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
# Create dataframe
dfNVDA = ticker_history(comp[0], dt)
dfAMD = ticker_history(comp[1], dt)

trainNVDA, testNVDA = train_test_split(dfNVDA, tSplit, col)
trainAMD, testAMD = train_test_split(dfAMD, tSplit, col)

trainStdNVDA = scale_fit_transform(trainNVDA, scaler)
trainStdAMD = scale_fit_transform(trainAMD, scaler)

XTrainNVDA, yTrainNVDA = split_time_series(trainStdNVDA, step)
XTrainAMD, yTrainAMD = split_time_series(trainStdAMD, step)

# reshape training data to fit on LSTM model
XTrainNVDA = XTrainNVDA.reshape(XTrainNVDA.shape[0], XTrainNVDA.shape[1], feature)
XTrainAMD = XTrainAMD.reshape(XTrainAMD.shape[0], XTrainAMD.shape[1], feature)

In [ ]:
lstmNVDA = model_lstm(step, feature)
lstmAMD = model_lstm(step, feature)

# train the model
lstmNVDA.fit(XTrainNVDA, yTrainNVDA, epochs=10, batch_size=32)
lstmAMD.fit(XTrainAMD, yTrainAMD, epochs=10, batch_size=32)

In [ ]:
XTestNVDA, yTestNVDA, yPredNVDA = predict_lstm(dfNVDA, lstmNVDA, testNVDA, XTrainNVDA, yTrainNVDA, col, step)
XTestAMD, yTestAMD, yPredAMD = predict_lstm(dfAMD, lstmAMD, testAMD, XTrainAMD, yTrainAMD, col, step)